![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)  ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)

# PEC M5: Procesamiento de datos con Apache Spark mediante RDDs (4 Dic 2024 - 31 Dic 2024)

# Contando palabras: Construye una aplicación que cuente palabras

En este laboratorio se trabajarán las tecnologías descritas en los materiales del curso sobre Spark para desarrollar una aplicación de conteo de palabras. 

Con el uso masivo de Internet y las redes sociales, el volumen de texto no estructurado está creciendo dramáticamente, y Spark es una gran herramienta para analizar este tipo de datos. En esta PEC, vamos a escribir código para encontrar las palabras más comunes en un archivo de texto con contenido de la biblia en inglés [Corpus Canterbury](https://corpus.canterbury.ac.nz/descriptions/).


Lo más interesante de la forma de trabajar en esta práctica es que podría escalarse para, por ejemplo, encontrar las palabras más comunes en Wikipedia.

## Durante esta PEC vamos a cubrir:

* *Parte 1:* Creación de un RDD y un pair RDD - **1.5 PUNTOS**
* *Parte 2:* Contar palabras usando un pair RDD - **2.5 PUNTOS**
* *Parte 3:* Encontrar las palabras individuales y su frecuencia de aparición media - **1 PUNTO**
* *Parte 4:* Aplicar las funcionalidades desarrolladas a un archivo de texto - **2 PUNTOS**
* *Parte 5:* Calcular algunos estadísticos - **3 PUNTOS**


> Como referencia a todos los detalles de los métodos que se usan en esta práctica usar:
> * [API Python de Spark](https://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD)

## Formato de entrega:

Para realizar la entrega, debéis subir los dos notebooks al apartado de evaluación, con el siguiente nombre:

* usuarioUOC_22.519_PEC1_WordCount_ES.ipynb



## Parte 1: Creacion de un RDD y un pair RDDs

En esta sección, exploraremos como crear RRDs usando `parallelize` y como aplicar pair RDDs al problema del conteo de palabras.

### (0) Configuración del entorno python + spark

In [1]:
import findspark
findspark.init()
import pyspark
import random
sc = pyspark.SparkContext(master="local[1]", appName="PAC1_sjuanmon")

### (1a) Creación de un RDD
Empecemos generando un RDD a partir de una lista de Python y el método `sc.parallelize`. Luego mostraremos por pantalla el tipo de la variable generada.

In [2]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)
# Print out the type of wordsRDD
type(wordsRDD)

pyspark.rdd.RDD

### (1b) Crear el plural de las palabras y testear

Vamos a utilizar una transformación `map()` para incorporar la letra 's' a cada uno de los strings almacenados en el RDD que acabamos de crear. Vamos a definir una función de Python que devuelva una palabra, que se le ha pasado como parámetro, incorporando una "s" al final de la misma. Reemplazar el texto `<FILL IN>` con la solución propuesta. después de haber definido correctamente la función `makePlural`, ejecutar la segunda celda que contiene un assert de test. Si la solución es correcta, se imprimira `1 test passed`.

Esta será la forma habitual de trabajar en las PECs. Los ejercicios contendrán una explicación de lo que se espera, seguido de una celda de Código con uno o más `<FILL IN>`. Las celdas que necesiten ser modificadas contendrán el texto `# TODO: Replace <FILL IN> with appropriate code` en la primera línea.

Una vez se hayan sustituido todos los `<FILL IN>` por el Código Python adecuado, ejecutar la celda, y posteriormente ejecutar la celda siguiente de test para comprobar que la solución es la esperada.


In [3]:
# TODO: Replace <FILL IN> with appropriate code
def makePlural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    #return <FILL IN>
    return word + 's' 

makePlural('cat')

'cats'

In [5]:
# TEST Pluralize and test (1b)
assert makePlural('rat') == 'rats', 'incorrect result: makePlural does not add an s'
print ("1 test passed")

1 test correcto


### (1c) Aplicar `makePlural` a nuestro RDD

Ahora es el momento de aplicar nuestra función `makePlural()` a todos los elementos del RDD usando una transformación [map()](https://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.map). Posteriormente ejecutar la acción [collect()](http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.collect) para obtener el RDD transformado.

In [6]:
# TODO: Replace <FILL IN> with appropriate code
pluralRDD = wordsRDD.map(makePlural)
pluralRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']

In [7]:
# TEST Apply makePlural to the base RDD(1c)
assert pluralRDD.collect() == ['cats', 'elephants', 'rats', 'rats', 'cats'], 'incorrect values for pluralRDD'

### (1d) Ejecutar una funcion `lambda` en un `map`

Vamos a crear el mismo RDD usando una `lambda` function en lugar de una función con nombre.

In [8]:
# TODO: Replace <FILL IN> with appropriate code
pluralLambdaRDD = wordsRDD.map(lambda word: word + 's')
print(pluralLambdaRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


In [9]:
# TEST Pass a lambda function to map (1d)
assert pluralLambdaRDD.collect() == ['cats', 'elephants', 'rats', 'rats', 'cats'], 'incorrect values for pluralLambdaRDD (1d)'

### (1e) Numero de caracteres de cada una de las palabras

Ahora vamos a usar un `map()` y una función lambda `lambda` para obtener el número de caracteres de cada palabra. Usaremos `collect` para guardar este resultado directamente en una variable.

In [10]:
# TODO: Replace <FILL IN> with appropriate code
pluralLengths = (pluralRDD.map(lambda word: len(word)).collect())
print(pluralLengths)

[4, 9, 4, 4, 4]


In [11]:
# TEST Length of each word (1e)
assert pluralLengths == [4, 9, 4, 4, 4], 'incorrect values for pluralLengths'

### (1f) Pair RDDs

El siguiente paso para completar nuestro programa de conteo de palabras en crear un nuevo tipo de RDD, llamado pair RDD. Un pair RDD es un RDD donde cada elemento es un tupla del estilo `(k, v)` donde `k` es la clave y `v` es su valor correspondiente. En este ejemplo, crearemos una pair RDD consistente en tuplas con el formato `('<word>', 1)` para cada elemento de nuestro RDD básico.

Podemos crear nuestro pair RDD usando una transformación `map()` con una `lambda()` function que cree un nuevo RDD.


In [12]:
# TODO: Replace <FILL IN> with appropriate code
wordPairs = wordsRDD.map(lambda word: (word, 1))
print(wordPairs.collect())

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


In [13]:
# TEST Pair RDDs (1f)
assert wordPairs.collect() == [('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)], 'incorrect value for wordPairs'

## Parte 2: Contar palabras usando un pair RDD

Ahora, contaremos el número de veces que una palabra en particular aparece en el RDD. Esta operación se puede realizar de una infinidad de maneras, pero algunas serán mucho menos eficientes que otras.

Un solución muy sencilla seria usar `collect()` sobre todos los elementos devolverlos al driver y allí contarlos. Mientras esta forma de trabajar podría funcionar con textos relativamente cortos, nosotros lo que queremos es poder trabajar con textos de cualquier longitud. Adicionalmente, ejecutar todo el cálculo en el driver es mucho más lento que ejecutarlo en paralelo en los workers. Por estos motivos, en esta práctica usaremos operaciones paralelizables.


### (2a) Usando `groupByKey()`
Una primera solución a nuestro problema, luego veremos que hay otras mucho más eficientes, se podría basar en la transformación [groupByKey()](http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.groupByKey). Como su nombre indica, la transformación `groupByKey()` agrupa todos los elementos de un RDD que compartan la misma clave en una única lista dentro de una de las particiones.

Esta operación plantea dos problemas:
  + Esta operación necesita mover todos los valores dentro de la partición adecuada. Esto satura la red. 
  + Las listas generadas pueden llegar a ser muy grandes llegando incluso a saturar la memoria de alguno de los trabajadores
  
Utiliza `groupByKey()` para generar un pair RDD del tipo `('word', iterator)`.


In [14]:
# TODO: Replace <FILL IN> with appropriate code
# Note that groupByKey requires no parameters
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print('{0}: {1}'.format(key, list(value)))

cat: [1, 1]
elephant: [1]
rat: [1, 1]


In [15]:
# TEST groupByKey() approach (2a)
assert sorted(wordsGrouped.mapValues(lambda x: list(x)).collect()) == [('cat', [1, 1]), ('elephant', [1]), ('rat', [1, 1])], 'incorrect value for wordsGrouped'

### (2b) Utiliza `groupByKey()` para obtener los conteos

Usando la transformación `groupByKey()` crea un RDD que contenga 2 elementos, donde cada uno de ellos sea un par palabra (clave) iterador de Python (valor).

Luego suma todos los valores de iterador usando una transformación `map()`. El resultado debe ser un pair RDD que contenga las parejas (word, count).


In [20]:
# TODO: Replace <FILL IN> with appropriate code
wordCountsGrouped = wordsGrouped.map(lambda x: (x[0], sum(x[1])))
print(wordCountsGrouped.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [21]:
# TEST Use groupByKey() to obtain the counts (2b)
assert sorted(wordCountsGrouped.collect())==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect value for wordCountsGrouped'


### (2c) Conteo usando `reduceByKey`

Una mejor solución es comenzar desde un pair RDD y luego usar la transformación [reduceByKey()](http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.reduceByKey) para crear un nuevo pair RDD. La transformación `reduceByKey()` agrupa todas las parejas que comparten la misma clave. Posteriormente aplica la funcion que se le pasa por parámetro agrupando los valores de dos en dos. Este proceso se repite iterativamente hasta que obtenemos un único valor agregado para cada una de las claves del pair RDD. `reduceByKey()` opera aplicando la función primero dentro de cada una de las particiones de forma independiente, y posteriormente únicamente comparte los valores agregados entre particiones diferentes, permitiéndole escalar de forma eficiente ya que no tiene necesidad de desplazar por la red una gran cantidad de datos.

In [16]:
# TODO: Replace <FILL IN> with appropriate code
# Note that reduceByKey takes in a function that accepts two values and returns a single value
wordCounts = wordPairs.reduceByKey((lambda a, b: a + b))
print (wordCounts.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [22]:
# TEST Counting using reduceByKey (2c)
assert sorted(wordCounts.collect())==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect value for wordCounts'

### (2d) Ahora todo junto

La versión más compleja del Código ejecuta primero un `map()` sobre el pair RDD, la transformación `reduceByKey()`, y finalmente la acción `collect()` en una única línea de Código.

In [23]:
# TODO: Replace <FILL IN> with appropriate code
wordCountsCollected = (wordsRDD
                       .map(lambda word: (word, 1))
                       .reduceByKey(lambda a, b: a + b)
                       .collect())
print(wordCountsCollected)

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [24]:
# TEST All together (2d)
assert sorted(wordCountsCollected)==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect value for wordCountsCollected'

## Parte 3: Encontrar las palabras individuales y su frecuencia de aparición media

### (3a) Palabras únicas

Calcular el número de palabras únicas en `wordsRDD`. Puedes utilizar otros RDDs que hayas creado en esta práctica si te resulta más sencillo.


In [25]:
# TODO: Replace <FILL IN> with appropriate code
uniqueWords = wordsRDD.distinct().count()
print(uniqueWords)

3


In [26]:
# TEST Unique words (3a)
assert uniqueWords== 3, 'incorrect count of uniqueWords'

### (3b) Calcular la media usando `reduce()`

Encuentra la frecuencia media de aparición de palabras en `wordCounts`.

Utiliza la acción `reduce()` para sumar los conteos en `wordCounts` y entonces divide por el número de palabras únicas. Para realizar esto primero aplica un `map()` al pair RDD `wordCounts`, que está formado por tuplas con el formato (key, value), para convertirlo en un RDD de valores.


In [27]:
# TODO: Replace <FILL IN> with appropriate code
from operator import add
totalCount = (wordCounts
              .map((lambda x: x[1]))
              .reduce(add))
average = totalCount / float(len(wordCounts.collect()))
print(totalCount)
print(round(average, 2))

5
1.67


In [28]:
# TEST Mean using reduce (3b)
assert round(average, 2)==1.67, 'incorrect value of average'

## Parte 4: Aplicar las funcionalidades desarrolladas a un archivo de texto

Para esto hemos de construir una función `wordCount`, capaz de trabajar con datos del mundo real que suelen presentan problemas como el uso de mayúsculas o minúsculas, puntuación, acentos, etc. Posteriormente, cargar los datos de nuestra fuente de datos y finalmente, calcular el conteo de palabras sobre los datos procesados.

### (4a) función `wordCount`

Primero, define una función para el conteo de palabras. deberías reutilizar las técnicas que has visto en los apartados anteriores de esta práctica. Dicha función, ha de tomar un RDD que contenga una lista de palabras, y devolver un pair RDD que contenga todas las palabras con sus correspondientes conteos.


In [29]:
# TODO: Replace <FILL IN> with appropriate code
# TODO: Replace <FILL IN> with appropriate code
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return wordListRDD \
        .map(lambda word: (word, 1)) \
        .reduceByKey(lambda a, b: a + b)
print(wordCount(wordsRDD).collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [30]:
# TEST wordCount function (4a)
assert sorted(wordCount(wordsRDD).collect())==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect definition for wordCount function'

### (4b) Mayúsculas y puntuación

Los ficheros del mundo real son mucho más complejos que los que hemos estado usando en esta PEC. Algunos de los problemas que son necesarios de solucionar son:
  + Las palabras deben de contarse independientemente de si están en mayúscula o minúscula (por ejemplo, Spark y spark deberían contarse como la misma palabra).
  + Todos los signos de puntuación han de eliminarse.
  + Cualquier espacio al principio o al final de la palabra ha de eliminarse.
  
Define la función `removePunctuation` que convierta todo el texto a minúsculas, elimine los signos de puntuación, y elimine los espacios al principio y fin de cada palabra. Usa el módulo de Python [re](https://docs.python.org/2/library/re.html) para eliminar cualquier carácter que no sea una letra, un numero o un espacio.

Sino estas familiarizado con las expresiones regulares deberías revisar [este tutorial](https://developers.google.com/edu/python/regular-expressions). Alternativamente, [esta web](https://regex101.com/#python) es de gran ayuda para debugar tus expresiones regulares.

**Hints**

1. Usa la función [re.sub()](https://docs.python.org/2.7/library/re.html#re.sub).
2. Para nuestros propósitos, "puntuación" significa "no alfabético, numérico, o espacio." La expresión regular que define estos caracteres es: `[^A-Za-z\s\d]`
3. No usar `\W`, ya que retendrá los guiones bajos.


In [31]:
# TODO: Replace <FILL IN> with appropriate code
import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only whitespace, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    # Eliminar caracteres no alfabéticos, numéricos ni espacios
    cleaned_text = re.sub(r'[^A-Za-z\s\d]', '', text)
    
    # Convertir a minúsculas y eliminar espacios al principio y al final
    cleaned_text = cleaned_text.lower().strip()
    
    return cleaned_text
    
print(removePunctuation('Hi, you!'))
print(removePunctuation(' No under_score!'))
print(removePunctuation(' *      Remove punctuation then spaces  * '))

hi you
no underscore
remove punctuation then spaces


In [32]:
# TEST Capitalization and punctuation (4b)
assert removePunctuation(" The Elephant's 4 cats. ") == 'the elephants 4 cats', 'incorrect definition for removePunctuation function'

### (4c) Cargar un archivo de texto

Para la siguiente parte, usaremos el fichero de la biblia en inglés ya mencionada. 
Para convertir un archivo de texto en un RDD, usaremos el método `SparkContext.textFile()`. También usaremos la función que acabamos de crear `removePunctuation()` dentro de una transformación `map()` para eliminar todos los caracteres no alfabéticos, numéricos o espacios. Dado que el archivo es bastante grande, usaremos `take(15)`, de forma que tan solo imprimiremos por pantalla las 15 primeras líneas


In [33]:
# Tan solo ejecuta este codigo
import os.path

fileName = os.path.join('/aula_B0.485/data', 'bible.txt')

bibleRDD = sc.textFile(fileName, 8).map(removePunctuation).filter(lambda x: len(x)>0)
bibleRDD.take(10)

['in the beginning god created the heaven and the earth and the earth was without form and void and darkness was upon the face of the deep and the spirit of god moved upon the face of the waters',
 'and god said let there be light and there was light',
 'and god saw the light that it was good and god divided the light from the darkness',
 'and god called the light day and the darkness he called night and the evening and the morning were the first day',
 'and god said let there be a firmament in the midst of the waters and let it divide the waters from the waters',
 'and god made the firmament and divided the waters which were under the firmament from the waters which were above the firmament and it was so',
 'and god called the firmament heaven and the evening and the morning were the second day',
 'and god said let the waters under the heaven be gathered together unto one place and let the dry land appear and it was so',
 'and god called the dry land earth and the gathering together o


### (4d) Extraer las palabras de las líneas

Antes de poder usar la función `wordcount()`, hemos de solucionar dos problemas con el formato del RDD:
  + El primer problema es que necesitamos dividir cada línea por sus espacios. ** Esto lo solucionaremos en el apartado (4d). **
  + El segundo problema es que necesitamos filtrar las líneas completamente vacías. ** Esto lo solucionaremos en el apartado (4e). **

Para aplicar una transformación que divida cada elemento del RDD por sus espacios, hemos de aplicar la función incorporada en los strings de Python [split()](https://docs.python.org/2/library/string.html#string.split). Cuidado que a primera vista puede parecer que la función necesaria es una transformación `map()`, pero si piensas un poco más sobre el resultado de la función `split()` te darás cuenta que esta no es la opción correcta.

> Nota:
> * No uséis la implementación estándar del `split()`, debéis pasar un valor de separación. Por ejemplo, para dividir `line` por comas, usa `line.split(',')`.


In [34]:
# TODO: Replace <FILL IN> with appropriate code
bibleWordsRDD = bibleRDD.flatMap(lambda line: line.split(' '))
bibleWordsCount = bibleWordsRDD.count()
print(bibleWordsRDD.top(5))
print(bibleWordsCount)

['zuzims', 'zurishaddai', 'zurishaddai', 'zurishaddai', 'zurishaddai']
766109


In [35]:
# TEST Words from lines (4d)
# This test allows for leading spaces to be removed either before or after
# punctuation is removed.
assert bibleWordsCount == 766109, 'incorrect value for bibleWordsCount'
assert bibleWordsRDD.top(5)==[u'zuzims', u'zurishaddai', u'zurishaddai', u'zurishaddai', u'zurishaddai'], 'incorrect value for bibleWordsRDD'

### (4e) Calcula palabras distintas

El siguiente paso es contar cuantas palabras distintas contiene nuestro texto. Puedes usar las transformaciones map() y reduceByKey() ya utilizadas anteriormente.

In [36]:
# TODO: Replace <FILL IN> with appropriate code

distintWordsMapRDD = bibleWordsRDD.map(lambda word: (word, 1))

distintWordsRDD=distintWordsMapRDD.keys().distinct()

print(distintWordsRDD.take(8))   
print(distintWordsRDD.count())

['', 'shiloh', 'tarsus', 'pharisees', 'tochen', 'avoided', 'babylonians', 'tents']
12606


In [37]:
# TEST Remove empty elements (4e)
assert distintWordsRDD.count()== 12606, 'incorrect value for distintWordsRDD'

### (4f) Cuenta las palabras

Ahora que tenemos un RDD que contiene solo palabras. El siguiente paso es aplicar la función `wordCount()` para producir una lista con los conteos de palabras. Podemos ver las 15 más comunes usando la acción `takeOrdered()`; sin embargo, como los elementos del RRD son pares, necesitamos una función especial que ordene los pares de la forma correcta.

Usad las funciones  `wordCount()` y `takeOrdered()` para obtener las 15 palabras más comunes junto con sus conteos.


In [38]:
# TODO: Replace <FILL IN> with appropriate code
top15WordsAndCounts = wordCount(bibleWordsRDD).takeOrdered(15, key = lambda x: -x[1])
print(top15WordsAndCounts)

[('the', 61680), ('and', 49862), ('of', 33195), ('to', 13031), ('that', 12561), ('in', 12211), ('he', 9939), ('shall', 9733), ('for', 8820), ('unto', 8808), ('i', 8712), ('his', 8141), ('a', 7997), ('lord', 7538), ('they', 7141)]


In [39]:
# TEST Count the words (4f)
assert top15WordsAndCounts== [('the', 61680), ('and', 49862), 
                              ('of', 33195), ('to', 13031), 
                              ('that', 12561), ('in', 12211), 
                              ('he', 9939), ('shall', 9733), 
                              ('for', 8820), ('unto', 8808), 
                              ('i', 8712), ('his', 8141), ('a', 7997), 
                              ('lord', 7538), ('they', 7141)],'incorrect value for top15WordsAndCounts'

## Parte 5: Calcular algunos estadísticos

Usando las mismas técnicas que has aplicado en los ejercicios anteriores responde a las siguientes preguntas:


### (5a) ¿Cuántas palabras distintas tienen exactamente 3 letras?
(Mostrad por pantalla una muestra)


In [40]:
threeLetterWordsRDD = bibleWordsRDD.filter(lambda word: len(word) == 3).distinct()

threeLetterWordsCount = threeLetterWordsRDD.count()

sampleThreeLetterWords = threeLetterWordsRDD.take(10)

print("Sample of 3-letter words:", sampleThreeLetterWords)
print("Total number of distinct 3-letter words:", threeLetterWordsCount)

Sample of 3-letter words: ['out', 'raw', 'wax', 'hem', 'yet', 'bid', 'eri', 'are', 'jaw', 'lap']
Total number of distinct 3-letter words: 301


### (5b) ¿Cuántas palabras distintas tienen en la tercera posición la vocal 'a'?
(Mostrad por pantalla una muestra de las palabras ordenadas por longitud decreciente)

In [47]:
# Filtrar las palabras con 'a' en la tercera posición
words_with_a_third_position = (
    bibleWordsRDD
    .filter(lambda word: len(word) >= 3 and word[2] == 'a')  # Filtrar palabras con 'a' en la tercera posición
    .distinct()  # Eliminar duplicados
)

# Contar cuántas palabras cumplen la condición
count = words_with_a_third_position.count()

# Obtener una muestra de palabras ordenadas por longitud decreciente
sample_sorted = (
    words_with_a_third_position
    .map(lambda word: (len(word), word))  # Crear pares (longitud, palabra)
    .sortByKey(ascending=False)  # Ordenar por longitud (descendente)
    .map(lambda x: x[1])  # Extraer solo las palabras
    .take(10)  # Tomar las primeras 10 palabras
)

# Mostrar los resultados
print("Total words with a in third position: {}".format(count))
print("Sample of words sorted by descending length: {}".format(sample_sorted))

Total words with a in third position: 1249
Sample of words sorted by descending length: ['maalehacrabbim', 'transgressions', 'grapegatherers', 'standardbearer', 'shamefacedness', 'grapegleanings', 'transgression', 'blasphemously', 'transgresseth', 'transgressest']


### (5c) ¿Cuáles son las 10 palabras más largas? De éstas, ¿cuántas tienen 3 o más vocales?
Mostradlas por pantalla

In [42]:
# 1. Obtener las 10 palabras más largas
top10LongestWords = bibleWordsRDD.map(lambda word: (word, len(word))) \
                                .distinct() \
                                .takeOrdered(10, key=lambda x: -x[1])

# 2. Filtrar palabras con 3 o más vocales
def count_vowels(word):
    vowels = 'aeiou'
    return sum(1 for char in word if char in vowels)

wordsWithThreeOrMoreVowels = bibleWordsRDD.filter(lambda word: count_vowels(word) >= 3).distinct()

# 3. Imprimir los resultados
print("Top 10 longest words:")
print([word for word, length in top10LongestWords])


# Mostramos las primeras 10 palabras con 3 o más vocales
print("\nWords with 3 or more vowels:")
print(wordsWithThreeOrMoreVowels.take(10))  

Top 10 longest words:
['mahershalalhashbaz', 'chushanrishathaim', 'unprofitableness', 'lovingkindnesses', 'covenantbreakers', 'selahammahlekoth', 'bashanhavothjair', 'kibrothhattaavah', 'chepharhaammonai', 'evilfavouredness']

Words with 3 or more vowels:
['remaining', 'eliada', 'jahdai', 'babylonians', 'createth', 'winebibber', 'benefactors', 'attired', 'desirable', 'costliness']
